In [1]:
import serial
import csv
import re
import dash
from dash.dependencies import Output, Input
import numpy as np
import pandas as pd
from dash import dcc
from dash import html
import plotly.graph_objs as go
from collections import deque
import time
from datetime import datetime
current_time = timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')

In [2]:
## Connecting to esp32c6

In [3]:
def serialConnection():
    try:
        serCon = serial.Serial("COM7",115200, timeout = 0)
        print("connection successful")
    except serial.SerialException:
        print("Check connection.Replug the board")
        time.sleep(20)
        serCon = serial.Serial("COM8",115200, timeout = 1)
    if not serCon.is_open:
        print("Switching to COM7 port")
        serCon = serial.Serial("COM7",115200,timeout = 1)
    return serCon

In [4]:
## Collecting Live CSI data
ser = serialConnection()

connection successful


In [5]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div(
    [
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1000,  # Update every 40 mili second
            n_intervals=0 # Update every 40 mili second
        ),
    ]
)

# Define the callback to update the graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('graph-update', 'n_intervals')]
)
def update_graph_live(n):
    x = np.arange(256)
    y = np.zeros(256)

    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        print(f"Received: {line}")  # Debugging line
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
        if re.match(r'^-?\d+(\s-?\d+)*$', line):
            data = [int(i) for i in line.split(' ')]
            print(f"Parsed Data: {data}")  # Debugging line

            fileName = f"CSI_{current_time}.csv"
            with open(fileName, "a", newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([timestamp] + data)
            y = np.array(data)

    data = go.Scatter(
        x=x,
        y=y,
        mode='lines+markers'
    )

    return {'data': [data], 'layout': go.Layout(xaxis=dict(range=[0, 256]),
                                                yaxis=dict(range=[-300, 300]),
                                                title='Live CSI Animation')}

if __name__ == '__main__':
    app.run_server(debug=True)